# Vehicle Detection: Building the Classifier: #


**This Jupyter notebook finds the optimal feature set, performs grid search for classifier parameters and then runs the classifer on scaled dataset.**

**The best classifier and scaler object are saved for use in detection (see 'Vehicle Detection' notebook). **



### Data Preparation ###

In [16]:
import os
import glob
import numpy as np
import cv2
import matplotlib.pyplot
from skimage.feature import hog
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [3]:
vehicle_dir = 'image_dataset/vehicles/'
non_vehicle_dir = 'image_dataset/non-vehicles/'

car_images = glob.glob(vehicle_dir + '**/*.png', recursive=True)
notcar_images = glob.glob(non_vehicle_dir + '**/*.png', recursive=True)

cars = []
notcars = []

for img in car_images:
    cars.append(img)
for img in notcar_images:
    notcars.append(img)    

###  Functions ###

In [4]:
### Taken from Udacity SDC lessons
def convert_color(img, conv='RGB2YCrCb'):
    if conv == 'RGB2YCrCb':
        return cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    if conv == 'BGR2YCrCb':
        return cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
    if conv == 'RGB2LUV':
        return cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
    if conv == 'BGR2LUV':
        return cv2.cvtColor(img, cv2.COLOR_BGR2LUV)
    if conv == 'BGR2HLS':
        return cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
    if conv == 'BGR2HSV':
        return cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

def get_hog_features(img, orient, pix_per_cell, cell_per_block, 
                        vis=False, feature_vec=True):
    # Call with two outputs if vis==True
    if vis == True:
        features, hog_image = hog(img, orientations=orient, 
                                  pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), 
                                  transform_sqrt=True, 
                                  visualise=vis, feature_vector=feature_vec, block_norm='L2')
        return features, hog_image
    # Otherwise call with one output
    else:      
        features = hog(img, orientations=orient, 
                       pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), 
                       transform_sqrt=True, 
                       visualise=vis, feature_vector=feature_vec, block_norm='L2')
        return features

def bin_spatial(img, size=(32, 32)):
    color1 = cv2.resize(img[:,:,0], size).ravel()
    color2 = cv2.resize(img[:,:,1], size).ravel()
    color3 = cv2.resize(img[:,:,2], size).ravel()
    return np.hstack((color1, color2, color3))
                        
def color_hist(img, nbins=32):    #bins_range=(0, 256)
    # Compute the histogram of the color channels separately
    channel1_hist = np.histogram(img[:,:,0], bins=nbins)
    channel2_hist = np.histogram(img[:,:,1], bins=nbins)
    channel3_hist = np.histogram(img[:,:,2], bins=nbins)
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    # Return the individual histograms, bin_centers and feature vector
    return hist_features



In [5]:
def single_img_features(img, color_space='BGR', spatial_size=(32, 32),
                        hist_bins=32, orient=9, 
                        pix_per_cell=8, cell_per_block=2, hog_channel='ALL',
                        spatial_feat=True, hist_feat=True, hog_feat=True):    
    #1) Define an empty list to receive features
    img_features = []
    #2) Apply color conversion if other than 'RGB'
    if color_space != 'BGR':
        if color_space == 'HSV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        elif color_space == 'LUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_BGR2LUV)
        elif color_space == 'HLS':
            feature_image = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
        elif color_space == 'YUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_BGR2YUV)
        elif color_space == 'YCrCb':
            feature_image = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
    else: feature_image = np.copy(img)      
    #3) Compute spatial features if flag is set
    if spatial_feat == True:
        spatial_features = bin_spatial(feature_image, size=spatial_size)
        #4) Append features to list
        img_features.append(spatial_features)
    #5) Compute histogram features if flag is set
    if hist_feat == True:
        hist_features = color_hist(feature_image, nbins=hist_bins)
        #6) Append features to list
        img_features.append(hist_features)
    #7) Compute HOG features if flag is set
    if hog_feat == True:
        if hog_channel == 'ALL':
            hog_features = []
            for channel in range(feature_image.shape[2]):
                hog_features.extend(get_hog_features(feature_image[:,:,channel], 
                                    orient, pix_per_cell, cell_per_block, 
                                    vis=False, feature_vec=True))      
        else:
            hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                        pix_per_cell, cell_per_block, vis=False, feature_vec=True)
        #8) Append features to list
        img_features.append(hog_features)

    #9) Return concatenated array of features
    return np.concatenate(img_features)

In [7]:
test_img = '../Plot3D_color_explore/images/53.jpg'
img = cv2.imread(test_img)

test_features = single_img_features(img)

In [8]:
test_features.shape


(8460,)

In [9]:
from sklearn.svm import SVC

class eSVC(SVC):
    def __init__(self, 
                
                color_space='BGR', spatial_size=(32, 32),
                hist_bins=32, orient=9, 
                pix_per_cell=8, cell_per_block=2, hog_channel='ALL',
                spatial_feat=True, hist_feat=True, hog_feat=True,
                
                C=1.0, kernel='rbf', degree=3, gamma='auto', 
                 coef0=0.0, shrinking=True, probability=False, 
                 tol=0.001, cache_size=200, class_weight=None, 
                 verbose=False, max_iter=-1, decision_function_shape='ovr', 
                 random_state=None):
        
        self.color_space=color_space
        self.spatial_size=spatial_size
        self.hist_bins=hist_bins 
        self.orient=orient
        self.pix_per_cell=pix_per_cell
        self.cell_per_block=cell_per_block 
        self.hog_channel=hog_channel
        self.spatial_feat=spatial_feat
        self.hist_feat=hist_feat
        self.hog_feat=hog_feat
        
        SVC.__init__(self, C=C, kernel=kernel, degree=degree, gamma=gamma, 
                 coef0=coef0, shrinking=shrinking, probability=probability, 
                 tol=tol, cache_size=cache_size, class_weight=class_weight, 
                 verbose=verbose, max_iter=max_iter, decision_function_shape=decision_function_shape, 
                 random_state=random_state)
        
    '''
    fit() takes an array of images as input and extracts features. The features are then fed into the classifier
    
    X: array-like, elements are BGR image arrays
    y: labels for each image
    sample-weights: passed on to SVC().fit()
    '''    
    def fit(self, X, y, sample_weight=None):
        X_features = []
        for x in X:
            X_features.append(single_img_features(x, color_space=self.color_space, spatial_size=self.spatial_size,
                hist_bins=self.hist_bins, orient=self.orient, 
                pix_per_cell=self.pix_per_cell, cell_per_block=self.cell_per_block, hog_channel=self.hog_channel,
                spatial_feat=self.spatial_feat, hist_feat=self.hist_feat, hog_feat=self.hog_feat))
        
        return SVC.fit(self, np.array(X_features), y)
    
    def predict(self, X):
        X_features = []
        
        if len(X.shape) == 3:
            X_features.append(single_img_features(X, color_space=self.color_space, spatial_size=self.spatial_size,
                hist_bins=self.hist_bins, orient=self.orient, 
                pix_per_cell=self.pix_per_cell, cell_per_block=self.cell_per_block, hog_channel=self.hog_channel,
                spatial_feat=self.spatial_feat, hist_feat=self.hist_feat, hog_feat=self.hog_feat))
        else:
            for x in X:
                X_features.append(single_img_features(x, color_space=self.color_space, spatial_size=self.spatial_size,
                    hist_bins=self.hist_bins, orient=self.orient, 
                    pix_per_cell=self.pix_per_cell, cell_per_block=self.cell_per_block, hog_channel=self.hog_channel,
                    spatial_feat=self.spatial_feat, hist_feat=self.hist_feat, hog_feat=self.hog_feat))
                
        return SVC.predict(self, np.array(X_features))
        
    def score(self, X, y, sample_weight=None):
        
        return SVC.score(self, X, y)
    
    

### Parameter Search ###
Using the above subclass and GridSearchCV from sklearn, we look for the best parametres for feature extraction:

In [269]:
### Create image arrays for use in GridSearchCV
X_sample_img = []
test_sample_size = 700
for i in range(test_sample_size):
    X_sample_img.append(cv2.imread(cars[i]))
for i in range(test_sample_size):
    X_sample_img.append(cv2.imread(notcars[i]))
X_sample_img = np.array(X_sample_img)
y = np.concatenate((np.ones(test_sample_size), np.zeros(test_sample_size))).astype(np.float32)

In [185]:
from sklearn.utils import shuffle
X_sample_img, y = shuffle(X_sample_img, y)

In [192]:
from sklearn.model_selection import GridSearchCV
param = {
    'color_space':['HLS', 'HSV', 'LUV'], 
    'spatial_size':[(32, 32)],
    'hist_bins':(32,16), 
    'orient':(9,12), 
    'pix_per_cell':(6,8), 
    'cell_per_block':(2,3), 
    'hog_channel':['ALL'],
                
    'C':(1.0, 1000.0), 
    'kernel':['linear'], 
    'gamma':(0.01, 100), 
    
}
clf_eSVC = eSVC()
grid_search = GridSearchCV(clf_eSVC, param_grid=param)

In [193]:
grid_search.fit(X_sample_img, y)

GridSearchCV(cv=None, error_score='raise',
       estimator=eSVC(C=1.0, cache_size=200, cell_per_block=2, class_weight=None, coef0=0.0,
   color_space='BGR', decision_function_shape='ovr', degree=3,
   gamma='auto', hist_bins=32, hist_feat=True, hog_channel='ALL',
   hog_feat=True, kernel='rbf', max_iter=-1, orient=9, pix_per_cell=8,
   probability=False, random_state=None, shrinking=True, spatial_feat=True,
   spatial_size=(32, 32), tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'kernel': ['linear'], 'hist_bins': (32, 16), 'pix_per_cell': (6, 8), 'cell_per_block': (2, 3), 'color_space': ['HLS', 'HSV', 'LUV'], 'gamma': (0.01, 100), 'orient': (9, 12), 'hog_channel': ['ALL'], 'C': (1.0, 1000.0), 'spatial_size': [(32, 32)]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [6]:
import pickle

In [195]:
gridscv = {'gscv': grid_search}

In [197]:
pickle.dump(gridscv, open('gridObject', 'wb'))

In [199]:
grid_search.best_params_

{'C': 1.0,
 'cell_per_block': 2,
 'color_space': 'HLS',
 'gamma': 0.01,
 'hist_bins': 16,
 'hog_channel': 'ALL',
 'kernel': 'linear',
 'orient': 9,
 'pix_per_cell': 6,
 'spatial_size': (32, 32)}

In [200]:
grid_search.best_score_

0.99357142857142855

In [203]:
param = {
    'color_space':['HLS', 'YUV'], 
    'spatial_size':[(32, 32), (64,64)],
    'hist_bins':[16], 
    'orient':[9], 
    'pix_per_cell':[6], 
    'cell_per_block':[2], 
    'hog_channel':['ALL'],
                
    'C':(1.0, 1000.0), 
    'kernel':['linear', 'rbf', 'sigmoid'], 
    'gamma':(0.01, 100), 
    
}
clf_eSVC_2 = eSVC()
grid_search_2 = GridSearchCV(clf_eSVC, param_grid=param)

In [204]:
grid_search_2.fit(X_sample_img, y)

GridSearchCV(cv=None, error_score='raise',
       estimator=eSVC(C=1.0, cache_size=200, cell_per_block=2, class_weight=None, coef0=0.0,
   color_space='BGR', decision_function_shape='ovr', degree=3,
   gamma='auto', hist_bins=32, hist_feat=True, hog_channel='ALL',
   hog_feat=True, kernel='rbf', max_iter=-1, orient=9, pix_per_cell=8,
   probability=False, random_state=None, shrinking=True, spatial_feat=True,
   spatial_size=(32, 32), tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'kernel': ['linear', 'rbf', 'sigmoid'], 'hist_bins': [16], 'pix_per_cell': [6], 'cell_per_block': [2], 'color_space': ['HLS', 'YUV'], 'gamma': (0.01, 100), 'orient': [9], 'hog_channel': ['ALL'], 'C': (1.0, 1000.0), 'spatial_size': [(32, 32), (64, 64)]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [205]:
grid_search_2.best_params_

{'C': 1.0,
 'cell_per_block': 2,
 'color_space': 'HLS',
 'gamma': 0.01,
 'hist_bins': 16,
 'hog_channel': 'ALL',
 'kernel': 'linear',
 'orient': 9,
 'pix_per_cell': 6,
 'spatial_size': (32, 32)}

In [213]:
grid_search_2.best_score_

0.99357142857142855

** NOTE: ** Several iterations of grid search were run in the same cells above, by varying the parameters

**The best parametres for *feature extraction* are: **

- cell_per_block: 2,
- color_space: 'HLS',
- hist_bins: 16,
- hog_channel: 'ALL',
- orient: 9,
- pix_per_cell: 6,
- spatial_size': (32, 32)

### Classifier and Parametre Search ###

Also from above, the best parametres for the **SVM classifier**, reporting a **best score of 0.99357** are:

- kernel: 'linear'
- C: 1.0

We find below that a **Random Forest Classifier** does a slightly better job albeit, **yeilding 0.995 score** with the following best parametres:

- criterion: 'entropy',
- min_samples_leaf: 1,
- min_samples_split: 2,
- n_estimators: 7

We'll have to explore both classifiers with the full dataset in the next section below.

In [286]:
rf_params = {
    'n_estimators':[7, 8, 10], 
    'criterion':['entropy'],  
    'min_samples_split':[2], 
    'min_samples_leaf':[1]
    
}
clf_rf = RandomForestClassifier()
rf_grid = GridSearchCV(clf_rf, rf_params)

In [271]:
X_sample = []
for x in X_sample_img:
    X_sample.append(single_img_features(x, color_space='HLS', spatial_size=(32,32),
    hist_bins=16, orient=9, 
    pix_per_cell=6, cell_per_block=2, hog_channel='ALL',
    spatial_feat=True, hist_feat=True, hog_feat=True))
X_sample = np.array(X_sample)

In [287]:
%time rf_grid.fit(X_sample, y)

Wall time: 4.29 s


GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'min_samples_split': [2], 'n_estimators': [7, 8, 10], 'min_samples_leaf': [1], 'criterion': ['entropy']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [289]:
rf_grid.best_params_

{'criterion': 'entropy',
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 8}

In [288]:
rf_grid.best_score_

0.98928571428571432

In [275]:
rf_grid.grid_scores_


C:\Users\admin\AppData\Local\conda\conda\envs\carnd-term1\lib\site-packages\sklearn\model_selection\_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.98429, std: 0.02067, params: {'min_samples_leaf': 1, 'min_samples_split': 2, 'criterion': 'gini', 'n_estimators': 7},
 mean: 0.98714, std: 0.01362, params: {'min_samples_leaf': 1, 'min_samples_split': 2, 'criterion': 'gini', 'n_estimators': 15},
 mean: 0.98929, std: 0.01061, params: {'min_samples_leaf': 1, 'min_samples_split': 2, 'criterion': 'gini', 'n_estimators': 20},
 mean: 0.97857, std: 0.02143, params: {'min_samples_leaf': 1, 'min_samples_split': 10, 'criterion': 'gini', 'n_estimators': 7},
 mean: 0.98429, std: 0.01921, params: {'min_samples_leaf': 1, 'min_samples_split': 10, 'criterion': 'gini', 'n_estimators': 15},
 mean: 0.98643, std: 0.01611, params: {'min_samples_leaf': 1, 'min_samples_split': 10, 'criterion': 'gini', 'n_estimators': 20},
 mean: 0.97643, std: 0.02280, params: {'min_samples_leaf': 20, 'min_samples_split': 2, 'criterion': 'gini', 'n_estimators': 7},
 mean: 0.98071, std: 0.01429, params: {'min_samples_leaf': 20, 'min_samples_split': 2, 'criterion': 'gi

## Training on all samples 
   (save test set)


In [8]:
X = []

### extract features for all images in the training dataset (cars, notcars) using the optimal parametres found above
for img_name in cars:
    img = cv2.imread(img_name)
    img_features = single_img_features(img, color_space='HLS', spatial_size=(32, 32),
                        hist_bins=16, orient=9, 
                        pix_per_cell=6, cell_per_block=2, hog_channel='ALL',
                        spatial_feat=True, hist_feat=True, hog_feat=True)
    X.append(img_features)
for img_name in notcars:
    img = cv2.imread(img_name)
    img_features = single_img_features(img, color_space='HLS', spatial_size=(32, 32),
                        hist_bins=16, orient=9, 
                        pix_per_cell=6, cell_per_block=2, hog_channel='ALL',
                        spatial_feat=True, hist_feat=True, hog_feat=True)
    X.append(img_features)

X = np.array(X)
y = np.concatenate((np.ones(len(cars)), np.zeros(len(notcars))))



In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

** Scale the features using the training set for transform fit: **

In [10]:
from sklearn.preprocessing import StandardScaler
X_scaler = StandardScaler()
X_scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)

In [11]:
X_test_scaled = X_scaler.transform(X_test)

#### Train and test Random Forest Classifier: ###

In [27]:
clf_randomForest = RandomForestClassifier(criterion='entropy', n_estimators=30, min_samples_leaf=1, min_samples_split=2)

%time clf_randomForest.fit(X_train_scaled_pca, y_train)

Wall time: 19.9 s


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=30, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [28]:
%time clf_randomForest.score(X_test_scaled_pca, y_test)

Wall time: 62.6 ms


0.96621621621621623

#### Train and test SVM classifier: ###

In [240]:
clf_svc = SVC(kernel='linear', C=1.0)
%time clf_svc.fit(X_train_scaled, y_train)

Wall time: 4min 32s


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [244]:
%time clf_svc.score(X_test_scaled, y_test)

Wall time: 1min 2s


0.99296171171171166

** Trying NB classifier** :

In [247]:
from sklearn.naive_bayes import GaussianNB
clf_nb = GaussianNB()
%time clf_nb.fit(X_train_scaled, y_train)

Wall time: 3.65 s


GaussianNB(priors=None)

In [248]:
%time clf_nb.score(X_test_scaled, y_test)

Wall time: 1.29 s


0.94538288288288286

Trying **AdaBoostClassifier**:

In [306]:
from sklearn.ensemble import AdaBoostClassifier
clf_ada = AdaBoostClassifier()
%time clf_ada.fit(X_train_scaled, y_train)

Wall time: 17min 9s


AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=None)

In [307]:
%time clf_ada.score(X_test_scaled, y_test)

Wall time: 1.77 s


0.98001126126126126

** Save the classifiers and the scaler: **

In [249]:
svc_scaler = {'clf_svc': clf_svc, 'X_scaler': X_scaler}
pickle.dump(svc_scaler, open('svc_scaler.p', 'wb'))

In [309]:
rf_scaler = {'clf_rf': clf_randomForest, 'X_scaler': X_scaler}
pickle.dump(rf_scaler, open('rf_scaler.p', 'wb'))

In [308]:
ada_scaler = {'clf_ada': clf_ada, 'X_scaler': X_scaler}
pickle.dump(ada_scaler, open('ada_scaler.p', 'wb'))

### Add PCA dimensionality reduction ###

The above SVM classifier was to slow in performing classification and thus we look into dimensionality reduction of feature set using Principal Component Ananlysis below:

In [31]:
from sklearn.decomposition import PCA

pca = PCA(n_components=600)
%time pca.fit(X_train_scaled)

Wall time: 1min 13s


PCA(copy=True, iterated_power='auto', n_components=600, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [32]:
%time X_train_scaled_pca = pca.transform(X_train_scaled)

Wall time: 2.79 s


In [33]:
%time X_test_scaled_pca = pca.transform(X_test_scaled)

Wall time: 1.12 s


In [34]:
clf_svc_pca = SVC(kernel='linear')
%time clf_svc_pca.fit(X_train_scaled_pca, y_train)

Wall time: 8.5 s


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [35]:
%time clf_svc_pca.score(X_test_scaled_pca, y_test)

Wall time: 1.15 s


0.98536036036036034

** Save the PCA object along with corresponding SVC and scaler:**

In [40]:
svc_dict = {'clf_svc_pca': clf_svc_pca, 'pca': pca, 'scaler': X_scaler }
pickle.dump(svc_dict, open('svc_pca_scaler.p', 'wb'))